In [8]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

from geometry_msgs.msg import PoseStamped

rospy.init_node("moveit_cartesian_path", anonymous=False)

rospy.loginfo("Starting node moveit_cartesian_path")

# Initialize the move_group API
moveit_commander.roscpp_initialize(sys.argv)

# Initialize the move group for the ur5_arm
arm = moveit_commander.MoveGroupCommander('manipulator')

# Get the name of the end-effector link
end_effector_link = arm.get_end_effector_link()

# Set the reference frame for pose targets
reference_frame = "/base_link"

# Set the ur5_arm reference frame accordingly
arm.set_pose_reference_frame(reference_frame)

# Allow replanning to increase the odds of a solution
arm.allow_replanning(True)

# Allow some leeway in position (meters) and orientation (radians)
arm.set_goal_position_tolerance(0.01)
arm.set_goal_orientation_tolerance(0.1)

# Get the current pose so we can add it as a waypoint
start_pose = arm.get_current_pose(end_effector_link).pose

# Initialize the waypoints list
waypoints = []

# Set the first waypoint to be the starting pose
# Append the pose to the waypoints list
waypoints.append(start_pose)

wpose = copy.deepcopy(start_pose)

# Set the next waypoint to the right 0.5 meters
wpose.position.y -= 0.1

waypoints.append(copy.deepcopy(wpose))

fraction = 0.0
maxtries = 100
attempts = 0

# Set the internal state to the current state
arm.set_start_state_to_current_state()

# Plan the Cartesian path connecting the waypoints
while fraction < 1.0 and attempts < maxtries:
    (plan, fraction) = arm.compute_cartesian_path (waypoints, 0.01, 0.0, True)

    # Increment the number of attempts
    attempts += 1

    # Print out a progress message
    if attempts % 10 == 0:
        rospy.loginfo("Still trying after " + str(attempts) + " attempts...")

# If we have a complete plan, execute the trajectory
if fraction == 1.0:
    rospy.loginfo("Path computed successfully. Moving the arm.")
    arm.execute(plan)
    rospy.loginfo("Path execution complete.")
else:
    rospy.loginfo("Path planning failed with only " + str(fraction) + " success after " + str(maxtries) + " attempts.")

arm.go()

[INFO] [1508165563.189122, 263.492000]: Starting node moveit_cartesian_path
[INFO] [1508165563.753397, 263.591000]: Path computed successfully. Moving the arm.
[INFO] [1508165563.762176, 264.153000]: Path execution complete.


False

In [9]:
help(arm)

Help on MoveGroupCommander in module moveit_commander.move_group object:

class MoveGroupCommander(__builtin__.object)
 |  Execution of simple commands for a particular group
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, robot_description='robot_description')
 |      Specify the group name for which to construct this commander instance. Throws an exception if there is an initialization error.
 |  
 |  allow_looking(self, value)
 |      Enable/disable looking around for motion planning
 |  
 |  allow_replanning(self, value)
 |      Enable/disable replanning
 |  
 |  attach_object(self, object_name, link_name='', touch_links=[])
 |      Given the name of an object existing in the planning scene, attach it to a link. The link used is specified by the second argument. If left unspecified, the end-effector link is used, if one is known. If there is no end-effector link, the first link in the group is used. If no link is identified, failure is reported. True is returned if an 

In [10]:
arm.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 314
    nsecs: 816000000
  frame_id: /world
pose: 
  position: 
    x: 0.816659732644
    y: 0.192372691339
    z: -0.00853823634231
  orientation: 
    x: -0.706702240274
    y: -0.707508423239
    z: 0.00137456605358
    w: 0.00137302835857